In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = None
import math
import requests
import sqlite3
import fantasy_utils as fu

In [2]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker, declarative_base
#from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [3]:
n_teams = 12
cols = ['cbsid', 'Name', 'Team', 'Pos', 'Owner', 'Keeper', 'Value', 'Paid', 'Timestamp']
pos_order = ['C', 'MI', 'CI', 'DH1', 'DH2', '2B', '3B', 'SS', '1B', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10']
owner_list = ["9 Grand Kids", 'Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lima Time', 'Mother', 'Roid Ragers', 'Trouble', 'Ugly Spuds', 'Wu-Tang', 'Young Guns']

In [61]:
df = pd.read_sql(f"SELECT * FROM players{datetime.now().year} WHERE cbsid IS NOT NULL", engine)
df.shape

(1442, 157)

In [62]:
owner = 'Lima Time'
mask = (df['Owner']==owner)

In [63]:
df.loc[mask][cols].sort_values('Timestamp')

,cbsid,Name,Team,Pos,Owner,Keeper,Value,Paid,Timestamp
22,1963334.0,Ketel Marte,ARI,2B,Lima Time,1.0,28.826732,11.0,2025-01-19 12:48:00
13,1953522.0,Marcell Ozuna,ATL,DH,Lima Time,1.0,31.743978,13.0,2025-01-20 12:48:00
140,2836763.0,Garrett Crochet,BOS,SP,Lima Time,1.0,8.056602,10.0,2025-01-21 12:48:00
253,28839811.0,Junior Caminero,TBR,3B,Lima Time,1.0,0.860272,0.0,2025-01-22 12:48:00
31,2044379.0,Christian Walker,HOU,1B,Lima Time,0.0,25.913066,18.0,2025-03-14 09:42:50.028482
45,2835063.0,Jake Burger,TEX,"1B,3B",Lima Time,0.0,20.326122,15.0,2025-03-14 09:43:12.431114
11,1894627.0,Francisco Lindor,NYM,SS,Lima Time,0.0,34.465496,34.0,2025-03-14 11:22:54.573184
222,2931974.0,Ryan Jeffers,MIN,C,Lima Time,0.0,3.077303,1.0,2025-03-14 11:27:03.434077
35,1765815.0,Christian Yelich,MIL,OF,Lima Time,0.0,24.846243,18.0,2025-03-14 11:29:54.684228
128,2910610.0,Isaac Paredes,HOU,"1B,3B",Lima Time,0.0,8.778436,10.0,2025-03-14 11:37:32.339960


In [47]:
def build_roster(n_teams, owner_list, df, pos_order):
    roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 
                             'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10'], data=np.zeros((33,n_teams)), columns=owner_list)
    for tm in owner_list:
        for i, row in df[df['Owner']==tm][['cbsid', 'Name', 'Owner', 'Pos', 'Paid', 'Supp', 'Team', 'Timestamp', 'Keeper', 'Value']].sort_values("Timestamp").iterrows():
            if row['Paid']==0:
                #print(df.loc[i]['Name'], 'B'+str(int(row['Supp'])))
                roster.loc['B'+str(int(row['Supp'])),tm] = row['Name']
                #fu.check_roster_pos(df.loc[i][cols].to_dict(), roster, df, 'B'+str(int(df.loc[i]['Supp'])), pos_order)
            else:
                if row['Paid'] > 0:
                    # Send info to check_roster_pos: Name, Owner, Primary_Pos, Eligible Pos list
                    #print(row['Name'], row['Timestamp'])
                    results = fu.check_roster_pos(row[cols].to_dict(), roster, df, pos_order)
                    print(results)
                    for result in results:
                        for item in result.items():
                            if item[1] != None:
                                roster.loc[item[1],tm] = item[0]
    return roster

In [48]:
roster = build_roster(n_teams, owner_list, df, pos_order)

[{'George Kirby': 'P1'}]
[{'Salvador Perez': 'C'}]
[{'Connor Wong': 'MI'}]
[{'Max Fried': 'P2'}]
[{'Vladimir Guerrero Jr.': 'CI'}]
[{'Chris Sale': 'P1'}]
[{'James Wood': 'DH1'}]
[{'Bailey Ober': 'P2'}]
[{'Shohei Ohtani': 'DH1'}]
[{'Logan Gilbert': 'P1'}]
[{'Spencer Strider': 'P2'}]
[{'Adley Rutschman': 'C'}]
[{'Robert Suarez': 'P1'}]
[{'Jose Miranda': 'CI'}]
[{'Gunnar Henderson': 'MI'}]
[{'Wyatt Langford': 'DH1'}]
[{'Michael Toglia': 'CI'}]
[{'Jordan Westburg': 'DH2'}]
[{'Ketel Marte': 'MI'}]
[{'Marcell Ozuna': 'DH1'}]
[{'Garrett Crochet': 'P1'}]
[{'Christian Walker': 'CI'}]
[{'Jake Burger': 'DH2'}]
[{'Francisco Lindor': 'SS'}]
[{'Ryan Jeffers': 'C'}]
[{'Christian Yelich': 'OF1'}]
[{'Isaac Paredes': '3B'}]
[{'Kerry Carpenter': 'OF2'}]
[{'Seiya Suzuki': 'OF3'}]
MI is currently used by Ketel Marte. Ketel Marte is also eligible at ['MI', 'DH1', 'DH2', '2B']
Ketel Marte is eligible to move to 2B. Moving Ketel Marte to 2B. Rostering Jeremy Pena in MI
[{'Jeremy Pena': 'MI'}, {'Ketel Marte': 

In [49]:
roster

,9 Grand Kids,Brewbirds,Charmer,Dirty Birds,Harvey,Lima Time,Mother,Roid Ragers,Trouble,Ugly Spuds,Wu-Tang,Young Guns
C,Salvador Perez,0.0,0.0,Adley Rutschman,0.0,Ryan Jeffers,0.0,0.0,0.0,0.0,0.0,William Contreras
1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2B,0.0,0.0,0.0,0.0,0.0,Ketel Marte,0.0,0.0,0.0,0.0,0.0,0.0
3B,0.0,0.0,0.0,0.0,0.0,Isaac Paredes,0.0,0.0,0.0,0.0,0.0,0.0
SS,0.0,0.0,0.0,0.0,0.0,Francisco Lindor,0.0,0.0,0.0,Corey Seager,0.0,0.0
MI,Connor Wong,0.0,0.0,0.0,Gunnar Henderson,Jeremy Pena,0.0,0.0,0.0,Zach Neto,Willy Adames,Jackson Holliday
CI,0.0,Vladimir Guerrero Jr.,0.0,Jose Miranda,Michael Toglia,Christian Walker,0.0,0.0,Matt Chapman,0.0,0.0,0.0
OF1,0.0,0.0,0.0,0.0,0.0,Christian Yelich,0.0,0.0,0.0,0.0,0.0,0.0
OF2,0.0,0.0,0.0,0.0,0.0,Kerry Carpenter,0.0,0.0,0.0,0.0,0.0,0.0
OF3,0.0,0.0,0.0,0.0,0.0,Seiya Suzuki,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df[df['Name']=='Bryce Harper'][cols].iloc[0].to_dict()

{'cbsid': 1765813.0,
 'Name': 'Bryce Harper',
 'Team': 'PHI',
 'Pos': '1B',
 'Owner': None,
 'Keeper': 0.0,
 'Value': 37.22367054961884,
 'Paid': nan,
 'Timestamp': None}

In [11]:
player = pd.read_sql(f"SELECT cbsid, Name, Owner FROM players{datetime.now().year} WHERE cbsid={1765813}", engine).iloc[0]
player['Owner'] = 'Lima Time'
player.to_dict()

{'cbsid': 1765813.0, 'Name': 'Bryce Harper', 'Owner': 'Lima Time'}

In [12]:
player_value = 30
team_max_bid = 0
max_bid = 35
np.random.uniform(player_value * 0.8, min(team_max_bid, max_bid))

7.013943820943354

In [13]:
min(team_max_bid, max_bid)

0

In [14]:
def sort_by_other_list(list_to_sort, order_list):
    """Sorts a list based on the order of another list."""

    order_dict = {value: index for index, value in enumerate(order_list)}
    return sorted(list_to_sort, key=lambda x: order_dict.get(x, float('inf')))
    
def check_roster_pos(roster, name, team_name, eligible, pos_order):
    """Create a legal roster of players by eligible positions

    roster: dataframe with index of all possible positions
    name: str, player name
    team_name: str, name of team drafting player
    eligible: str, comma-separated positions the player is eligibile to play
    pos_order: list of positions sorted by the order to try to place the player
    """
    # Establish empty list
    eligibility = []
    # Split into eligible positions and extend by associated positions
    for position in eligible.split(','):
        if position=='C':
            eligibility.extend(['C', 'DH1', 'DH2'])
        if position=='1B':
            eligibility.extend(['1B', 'CI', 'DH1', 'DH2'])
        if position=='2B':
            eligibility.extend(['2B', 'MI', 'DH1', 'DH2'])
        if position=='3B':
            eligibility.extend(['3B', 'CI', 'DH1', 'DH2'])
        if position=='SS':
            eligibility.extend(['SS', 'MI', 'DH1', 'DH2'])
        if position=='OF':
            eligibility.extend(['OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2'])
        if position=='DH':
            eligibility.extend(['DH1', 'DH2'])
        if position in ['SP', 'RP', 'P']:
            eligibility.extend(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'])
        if position in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10']:
            eligibility.extend([position])

    # Remove any duplicate positions
    eligibility = list(set(list(dict.fromkeys(eligibility))))
    # Sort by the pos_order list 
    eligibility = sort_by_other_list(eligibility, pos_order)
    #print(eligibility)
    for position in eligibility:
        # Check to see if the position is taken already
        #print(position, roster.loc[position, team_name])
        if roster.loc[position, team_name]==0:
            # If position is available, put the team there
            roster.loc[position, team_name] = name
            return position
    # If all eligibile positions taken, try bumping other players
    # Code here
    return eligibility

In [15]:
import itertools

def get_eligible_positions(pos, pos_order):
    # Expand player eligibility 
    eligibility = []
    for position in pos.split(','):
        if position=='C':
            eligibility.extend(['C', 'DH1', 'DH2'])
        if position=='1B':
            eligibility.extend(['1B', 'CI', 'DH1', 'DH2'])
        if position=='2B':
            eligibility.extend(['2B', 'MI', 'DH1', 'DH2'])
        if position=='3B':
            eligibility.extend(['3B', 'CI', 'DH1', 'DH2'])
        if position=='SS':
            eligibility.extend(['SS', 'MI', 'DH1', 'DH2'])
        if position=='OF':
            eligibility.extend(['OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2'])
        if position=='DH':
            eligibility.extend(['DH1', 'DH2'])
        if position in ['SP', 'RP', 'P']:
            eligibility.extend(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'])
        if position in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10']:
            eligibility.extend([position])
    
    # Remove duplicates and sort
    eligibility = list(set(list(dict.fromkeys(eligibility))))
    eligibility = sort_by_other_list(eligibility, pos_order)
    return eligibility



def _check_roster_pos(roster, name, team_name, eligible, pos_order):
    """
    Place a player on the roster by finding a valid permutation of positions
    
    Args:
    roster: DataFrame with index of all possible positions
    name: str, player name
    team_name: str, name of team drafting player
    eligible: str, comma-separated positions the player is eligible to play
    pos_order: list of positions sorted by the order to try to place the player
    
    Returns:
    str or None: Assigned position, or None if no placement possible
    """
    eligibility = get_eligible_positions(eligible, pos_order)
    
    # Get current team roster
    team_roster = roster[team_name]
    
    # Find current players and their original positions
    current_players = team_roster[team_roster != 0]
    
    # Get all current rostered positions
    rostered_positions = current_players.index.tolist()
    
    # Combine current players with new player
    all_players = list(current_players.values) + [name]

    # First check rostered_positions for a free spot
    for position in eligibility:
        if position not in rostered_positions:
            roster.loc[position, team_name] = name
            # End function looping b/c we found a place to put the player
            return position

    if bump_player(roster, rostered_positions, eligibility, pos_order):
        return change_made
    else:
        return f'Unable to roster {name}'


In [16]:
def bump_player(roster, rostered_positions, eligibility, pos_order):
    change_made = False
    for p in eligibility:
        used_pos_player_name = roster.loc[p,team_name]
        used_pos_player_positions = get_eligible_positions(df[df['Name']==used_pos_player_name]['Pos'].iloc[0], pos_order)
        print(f"{p} is currently used by {used_pos_player_name}. {used_pos_player_name} is also eligible at {used_pos_player_positions}")
        for p2 in used_pos_player_positions:
            if p2 not in rostered_positions:
                print(f"{used_pos_player_name} is eligible to move to {p2}. Moving {used_pos_player_name} to {p2}")
                roster.loc[p2, team_name] = used_pos_player_name
                roster.loc[p, team_name] = name
                change_made = True
            if change_made == True:
                break
        if change_made == True:
            break
        
    if change_made == False:
        print('\nCould not find a spot for this player')
    return change_made

In [17]:
get_eligible_positions(df[df['Name']=='Yordan Alvarez']['Pos'].iloc[0], pos_order)

['DH1', 'DH2', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5']

In [24]:
name = 'Freddie Freeman'
#name = 'Isaac Paredes'
#name = 'Jeremy Pena'
#name = 'Mookie Betts'
#name = 'Brandon Lowe'
#name = 'Jake Meyers'
#name = 'Jackson Chourio'
name = 'Max Kepler'
#name = 'Gunnar Henderson'
#name = 'Isaac Paredes'
eligible = df[df['Name']==name]['Pos'].iloc[0]
team_name = 'Lima Time'
_check_roster_pos(roster, name, team_name, eligible, pos_order)

DH1 is currently used by Marcell Ozuna. Marcell Ozuna is also eligible at ['DH1', 'DH2']
DH2 is currently used by Jake Burger. Jake Burger is also eligible at ['CI', 'DH1', 'DH2', '3B', '1B']
Jake Burger is eligible to move to 1B. Moving Jake Burger to 1B


NameError: name 'change_made' is not defined

In [19]:
roster

,9 Grand Kids,Brewbirds,Charmer,Dirty Birds,Harvey,Lima Time,Mother,Roid Ragers,Trouble,Ugly Spuds,Wu-Tang,Young Guns
C,Salvador Perez,0.0,0.0,Adley Rutschman,0.0,Ryan Jeffers,0.0,0.0,0.0,0.0,0.0,William Contreras
1B,0.0,0.0,0.0,0.0,0.0,Freddie Freeman,0.0,0.0,0.0,0.0,0.0,0.0
2B,0.0,0.0,0.0,0.0,0.0,Ketel Marte,0.0,0.0,0.0,0.0,0.0,0.0
3B,0.0,0.0,0.0,0.0,0.0,Isaac Paredes,0.0,0.0,0.0,0.0,0.0,0.0
SS,0.0,0.0,0.0,0.0,0.0,Francisco Lindor,0.0,0.0,0.0,Corey Seager,0.0,0.0
MI,Connor Wong,0.0,0.0,0.0,Gunnar Henderson,Jeremy Pena,0.0,0.0,0.0,Zach Neto,Willy Adames,Jackson Holliday
CI,0.0,Vladimir Guerrero Jr.,0.0,Jose Miranda,Michael Toglia,Christian Walker,0.0,0.0,Matt Chapman,0.0,0.0,0.0
OF1,0.0,0.0,0.0,0.0,0.0,Christian Yelich,0.0,0.0,0.0,0.0,0.0,0.0
OF2,0.0,0.0,0.0,0.0,0.0,Kerry Carpenter,0.0,0.0,0.0,0.0,0.0,0.0
OF3,0.0,0.0,0.0,0.0,0.0,Seiya Suzuki,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
#name = 'Isaac Paredes'
name = 'Mookie Betts'
name = 'Gleyber Torres'
name = 'Max Fried'
eligible = df[df['Name']==name]['Pos'].iloc[0]
team_name = 'Lima Time'

eligibility = get_eligible_positions(eligible, pos_order)
    
# Get current team roster
team_roster = roster[team_name]

# Find current players and their original positions
current_players = team_roster[team_roster != 0]

# Get all current rostered positions
rostered_positions = current_players.index.tolist()

# Combine current players with new player
all_players = list(current_players.values) + [name]

In [54]:
eligibility, rostered_positions

(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'],
 ['C',
  '2B',
  '3B',
  'SS',
  'MI',
  'CI',
  'OF1',
  'OF2',
  'OF3',
  'OF4',
  'OF5',
  'DH1',
  'DH2',
  'P1',
  'P2',
  'P3',
  'P4',
  'P5',
  'P6',
  'P7',
  'P8',
  'P9',
  'B1'])

In [51]:
for position in eligibility:
    if position not in rostered_positions:
        print(position)
        print([{'Name':position}])

P9
[{'Name': 'P9'}]


In [52]:
check_roster_pos(roster, 'Max Fried', 'Lima Time', eligible, pos_order)

['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']